In [84]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly
from numpy.fft import fft, fftfreq, ifft, rfft

stopper_data = pd.read_csv('./data/all/_all_Stopper_80_to_180.csv', header=None)
wall_data = pd.read_csv('./data/all/_all_Wall_Collection_80_to_180.csv', header=None)

In [85]:
def fft_from_data_frame(data_frame):
    fs= 1e6
    signal_set = []
    for index, row in data_frame.iterrows():
        fft_data = fft(row, n=row.size)/row.size
        freq = fftfreq(row.size, d=1/fs)
        cut_high_signal = np.abs(fft_data).copy()
        cut_high_signal[(freq > 50000)] = 0
        cut_high_signal[(freq < 30000)] = 0
        signal_set.append(np.abs(cut_high_signal))
    return signal_set

In [87]:
stopper_fft = fft_from_data_frame(stopper_data)
wall_fft = fft_from_data_frame(wall_data)
X = stopper_fft + wall_fft
y = [1]*len(stopper_fft) + [2]*len(wall_fft)
X = np.array(X)
y = np.array(y)

In [99]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn import metrics

# kf = KFold(n_splits=8)
kf = KFold(n_splits=4, shuffle=True)
from sklearn import metrics

for i in range(1,10):
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(i,), random_state=1)
    score = np.array([])
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf.fit(X_train,y_train)
        y_clf_result = clf.predict(X_test)
        score = np.append(score, [accuracy_score(y_clf_result, y_test)])
        pred = clf.predict_proba(X_test)[::,1]
    print("hidden layer:",i, "and accuracy score: ",np.average(score))

hidden layer: 1 and accuracy score:  0.9885386819484241
hidden layer: 2 and accuracy score:  0.9820916905444127
hidden layer: 3 and accuracy score:  0.9756446991404011
hidden layer: 4 and accuracy score:  0.9720630372492836
hidden layer: 5 and accuracy score:  0.979942693409742
hidden layer: 6 and accuracy score:  0.5279369627507163
hidden layer: 7 and accuracy score:  0.9842406876790831
hidden layer: 8 and accuracy score:  0.9799426934097422
hidden layer: 9 and accuracy score:  0.9749283667621778


In [100]:
from sklearn.model_selection import cross_val_score

for i in range(1,10):
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(i,), random_state=1)
    scores = cross_val_score(clf, X, y, cv=4, scoring="accuracy")
    print("hidden layer", i, "score average", scores.mean())

hidden layer 1 score average 0.9112131306430289
hidden layer 2 score average 0.934852287323387
hidden layer 3 score average 0.9411638930841578
hidden layer 4 score average 0.9584114123863161
hidden layer 5 score average 0.954790675681398
hidden layer 6 score average 0.5279370803750806
hidden layer 7 score average 0.9405193703803031
hidden layer 8 score average 0.94624184275034
hidden layer 9 score average 0.967032090279052
